In [24]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder


def load_data(data_dir):
    data = []
    labels = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            frames = []
            for file in sorted(os.listdir(label_dir)):
                if file.endswith('.npy'):
                    frame_data = np.load(os.path.join(label_dir, file))
                    frames.append(frame_data)
            data.append(frames)
            labels.append(label)
    return data, labels

data_dir = 'data'  # Adjust this path
data, labels = load_data(data_dir)

def normalize_data(data):
    normalized_data = []
    for sequence in data:
        norm_sequence = []
        for frame in sequence:
            norm_frame = (frame - np.min(frame, axis=0)) / (np.max(frame, axis=0) - np.min(frame, axis=0))
            norm_sequence.append(norm_frame)
        normalized_data.append(norm_sequence)
    return normalized_data

normalized_data = normalize_data(data)


def pad_truncate_sequences(data, max_len):
    padded_data = tf.keras.preprocessing.sequence.pad_sequences(data, maxlen=max_len, dtype='float32', padding='post', truncating='post')
    return padded_data

max_sequence_length = 150  # Set this to a reasonable number based on your data
padded_data = pad_truncate_sequences(normalized_data, max_sequence_length)


label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
np.save('classes.npy', label_encoder.classes_)

X_train = padded_data
y_train = encoded_labels


print(f'Training data shape: {X_train.shape}')
print(f'Training labels shape: {y_train.shape}')


Training data shape: (8, 150, 21, 3)
Training labels shape: (8,)


In [25]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, LSTM, Input

# Reshape the data to (batch_size, timesteps, features)
X_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], -1))

# Define the LSTM model
model = tf.keras.models.Sequential([
    Input(shape=(X_reshaped.shape[1], X_reshaped.shape[2])),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model on the entire dataset
history = model.fit(X_reshaped, y_train, epochs=20)



Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.1250 - loss: 2.0706
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.0000e+00 - loss: 2.0361
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.2500 - loss: 2.0026
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.2500 - loss: 1.9702
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.2500 - loss: 1.9391
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.2500 - loss: 1.9105
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.2500 - loss: 1.8844
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.2500 - loss: 1.8638
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.2500 - loss: 1.8488
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.2500 - loss: 1.8379
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.2500 - loss: 1.8294
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.2500 - l

In [26]:
# Save the model in the recommended format
model.save('sign_language_model.keras')

print("Model training completed and saved as 'sign_language_model.keras'")

Model training completed and saved as 'sign_language_model.keras'
